In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, glob, pickle, time, gc, copy, sys
import warnings
import cv2
import multiprocessing
from sklearn import metrics

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
sys.path.append('../src')
from utils import ri, pickle_load, pickle_save
from preprocess_fold import data_split_StratifiedKFold

# Config

In [2]:
# config
input_dir = "../input/orig"
preprocess_dir = "../input/preprocess"
df_train_path = "{}/df_train.csv".format(preprocess_dir)
load_dir_b0 = "../output"
pred_name_b0 = "cnn_preds_b0"
load_dir_b2 = "../output"
pred_name_b2 = "cnn_preds_b2"
NUM_FOLD = 5

In [3]:
col_index = 'SOPInstanceUID'
col_groupby = 'StudyInstanceUID'
col_targets = [
    'negative_exam_for_pe',
    'indeterminate',
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',
    'pe_present_on_image',
]
NUM_CLASS = len(col_targets)
print('NUM_CLASS: {}'.format(NUM_CLASS))

NUM_CLASS: 10


# Data loading

In [4]:
# load train data
df_train = pd.read_csv(df_train_path)
print(df_train.shape)
df_train.head()

(1790594, 27)


,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
1,6897fa9de148,2bfbb7fd2e8b,52b6b0b793bb,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,1.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
2,6897fa9de148,2bfbb7fd2e8b,1997c99c9d59,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,2.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
3,6897fa9de148,2bfbb7fd2e8b,c6f29ac6659b,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,3.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
4,6897fa9de148,2bfbb7fd2e8b,487d9ab5531f,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,4.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0


In [5]:
# make exam-level train data
df_train_exam = ri(df_train[df_train[col_groupby].duplicated()==False])
print(df_train_exam.shape)
df_train_exam.head()

(7279, 27)


,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.338710,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0
1,013358b540bb,2805267980e7,c0e16bbe96ef,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,145,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,1
2,0cee26703028,bac7becd2970,64bf37f1a302,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,144,17.0,0.118056,/mnt/disks/data6/rsna2020/preprocessed/train_l...,2
3,c28f3d01b14f,7d17c72fd0ce,265035a0f7aa,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,105,6.0,0.057143,/mnt/disks/data6/rsna2020/preprocessed/train_l...,3
4,c8fbf1e08ac5,275497911f02,7a33bbba8386,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,81,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,4


In [6]:
# data splitting stratified by 3 classes, PE positive, PE negative or indeterminate
df_tmp = df_train_exam
df_tmp['3class'] = 1 # PE positive
df_tmp['3class'][df_tmp['negative_exam_for_pe']==1] = 0 # PE negative
df_tmp['3class'][df_tmp['indeterminate']==1] = 2 # indeterminate
df_fold_exam = data_split_StratifiedKFold(df_tmp, col_groupby, col_stratified='3class') # apply stratified K-fold
df_fold = pd.merge(df_train[[col_groupby]], df_fold_exam, on=col_groupby, how='left')
print(df_fold.shape)
print(df_fold_exam.shape)
df_fold_exam.head()

(1790594, 11)
(7279, 11)


,StudyInstanceUID,fold1_train,fold1_valid,fold2_train,fold2_valid,fold3_train,fold3_valid,fold4_train,fold4_valid,fold5_train,fold5_valid
0,6897fa9de148,1,0,0,1,1,0,1,0,1,0
1,013358b540bb,0,1,1,0,1,0,1,0,1,0
2,0cee26703028,1,0,1,0,0,1,1,0,1,0
3,c28f3d01b14f,1,0,1,0,1,0,1,0,0,1
4,c8fbf1e08ac5,0,1,1,0,1,0,1,0,1,0


In [7]:
# load exam-level prediction
# load predict of EfficientNet-B0
pred_valid = np.zeros([len(df_train_exam), 9], np.float32)
for fold in range(NUM_FOLD):
    pred_valid[df_fold_exam['fold{}_valid'.format(fold+1)]==1] =\
        np.load("{}/{}_valid1_fold{}.npy".format(load_dir_b0, pred_name_b0, fold+1))
for i, col in enumerate(col_targets[:-1]):
    df_train_exam['{}_pred_b0'.format(col)] = pred_valid[:,i]
# load predict of EfficientNet-B2
for fold in range(NUM_FOLD):
    pred_valid[df_fold_exam['fold{}_valid'.format(fold+1)]==1] =\
        np.load("{}/{}_valid1_fold{}.npy".format(load_dir_b2, pred_name_b2, fold+1))
for i, col in enumerate(col_targets[:-1]):
    df_train_exam['{}_pred_b2'.format(col)] = pred_valid[:,i]
df_train_exam.head()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index,3class,negative_exam_for_pe_pred_b0,indeterminate_pred_b0,chronic_pe_pred_b0,acute_and_chronic_pe_pred_b0,central_pe_pred_b0,leftsided_pe_pred_b0,rightsided_pe_pred_b0,rv_lv_ratio_gte_1_pred_b0,rv_lv_ratio_lt_1_pred_b0,negative_exam_for_pe_pred_b2,indeterminate_pred_b2,chronic_pe_pred_b2,acute_and_chronic_pe_pred_b2,central_pe_pred_b2,leftsided_pe_pred_b2,rightsided_pe_pred_b2,rv_lv_ratio_gte_1_pred_b2,rv_lv_ratio_lt_1_pred_b2
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.338710,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0,1,0.005206,0.000391,0.043231,0.068667,0.133790,0.831051,0.940606,0.569271,0.487858,0.008044,0.000486,0.024745,0.059922,0.164815,0.843225,0.940056,0.331386,0.715898
1,013358b540bb,2805267980e7,c0e16bbe96ef,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,145,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,1,0,0.862018,0.025074,0.026926,0.002539,0.001570,0.057617,0.084721,0.021314,0.081259,0.713387,0.037043,0.066327,0.016371,0.015283,0.154356,0.208068,0.086355,0.201414
2,0cee26703028,bac7becd2970,64bf37f1a302,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,144,17.0,0.118056,/mnt/disks/data6/rsna2020/preprocessed/train_l...,2,1,0.038332,0.006295,0.060682,0.107211,0.169563,0.714764,0.844691,0.368361,0.609869,0.036494,0.003836,0.089072,0.125033,0.296316,0.785155,0.879427,0.538470,0.404576
3,c28f3d01b14f,7d17c72fd0ce,265035a0f7aa,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,105,6.0,0.057143,/mnt/disks/data6/rsna2020/preprocessed/train_l...,3,1,0.683958,0.091796,0.074759,0.022687,0.023941,0.183464,0.212422,0.055895,0.247339,0.760737,0.025251,0.041022,0.010597,0.010542,0.118750,0.177982,0.075317,0.155517
4,c8fbf1e08ac5,275497911f02,7a33bbba8386,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,81,0.0,0.000000,/mnt/disks/data6/rsna2020/preprocessed/train_l...,4,0,0.596162,0.026290,0.073517,0.022031,0.020258,0.213805,0.325244,0.139950,0.264076,0.781101,0.042583,0.043091,0.008422,0.010091,0.104869,0.129723,0.048102,0.128720


In [8]:
# load image-level prediction
# load predict of EfficientNet-B0
pred_valid = np.zeros([len(df_train), 2], np.float32)
for fold in range(NUM_FOLD):
    pred_valid[df_fold['fold{}_valid'.format(fold+1)]==1, 0] =\
            np.load("{}/{}_valid2_fold{}.npy".format(load_dir_b0, pred_name_b0, fold+1))[:,0]
    pred_valid[df_fold['fold{}_valid'.format(fold+1)]==1, 1] =\
            np.load("{}/{}_valid3_fold{}.npy".format(load_dir_b0, pred_name_b0, fold+1))[:,0]
df_train['{}_pred_b0'.format(col_targets[-1])] = pred_valid[:,0]
df_train['{}_qpred_b0'.format(col_targets[-1])] = pred_valid[:,1]
# load predict of EfficientNet-B2
for fold in range(NUM_FOLD):
    pred_valid[df_fold['fold{}_valid'.format(fold+1)]==1, 0] =\
            np.load("{}/{}_valid2_fold{}.npy".format(load_dir_b2, pred_name_b2, fold+1))[:,0]
    pred_valid[df_fold['fold{}_valid'.format(fold+1)]==1, 1] =\
            np.load("{}/{}_valid3_fold{}.npy".format(load_dir_b2, pred_name_b2, fold+1))[:,0]
df_train['{}_pred_b2'.format(col_targets[-1])] = pred_valid[:,0]
df_train['{}_qpred_b2'.format(col_targets[-1])] = pred_valid[:,1]
df_train.head()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,dicom_path,RescaleSlope,RescaleIntercept,PatientPosition,series_index,num_series,m_i,q_i,npy_path,exam_index,pe_present_on_image_pred_b0,pe_present_on_image_qpred_b0,pe_present_on_image_pred_b2,pe_present_on_image_qpred_b2
0,6897fa9de148,2bfbb7fd2e8b,baedb900c69c,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,0.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0,0.005252,0.006382,0.000906,0.001386
1,6897fa9de148,2bfbb7fd2e8b,52b6b0b793bb,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,1.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0,0.005291,0.006360,0.000785,0.001207
2,6897fa9de148,2bfbb7fd2e8b,1997c99c9d59,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,2.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0,0.005574,0.006727,0.000684,0.001063
3,6897fa9de148,2bfbb7fd2e8b,c6f29ac6659b,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,3.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0,0.005412,0.006857,0.000679,0.001053
4,6897fa9de148,2bfbb7fd2e8b,487d9ab5531f,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,/mnt/disks/data4/rsna2020/preprocessed/train-j...,1.0,-1024.0,HFS,4.0,124,42.0,0.33871,/mnt/disks/data6/rsna2020/preprocessed/train_l...,0,0.004932,0.006007,0.001084,0.001693


# Score Calculation

In [9]:
def calc_metric(y_true_exam, pred_exam, y_true_imag, pred_imag, q_image):
    weights = np.array([
        0.0736196319, 
        0.09202453988, 
        0.1042944785, 
        0.1042944785, 
        0.1877300613, 
        0.06257668712, 
        0.06257668712,
        0.2346625767,
        0.0782208589,
        0.07361963,
    ])
    score_list = []
    scores = {}
    for i in range(9):
        bce = metrics.log_loss(y_true_exam[:,i], pred_exam[:,i])
        auc = metrics.roc_auc_score(y_true_exam[:,i], pred_exam[:,i])
        scores[col_targets[i]] = bce
        score_list.append(bce)
        scores["{}_auc".format(col_targets[i])] = auc
        print("{} bce: {:.6f}, auc: {:.6f}".format((col_targets[i]+" "*50)[:30], bce, auc))
    score_s = np.sum(weights[:-1]*np.array(score_list))
    
    scores["weighted_average_study_bce"] = score_s
    
    print("{} bce: {:.6f}".format(("weighted_average_study_bce" +" "*50)[:30], score_s))
    score_i =  np.sum(- q_image * (y_true_imag*np.log(pred_imag) + (1-y_true_imag)*np.log(1-pred_imag))) / np.sum(q_image)
    auc = metrics.roc_auc_score(y_true_imag, pred_imag)
    scores["q_weighted_average_image_bce"] = score_i
    scores["image_auc"] = auc
    print("{} bce: {:.6f}, auc: {:.6f}".format(("q_weighted_average_image_bce" +" "*50)[:30], score_i, auc))
    score_all = (score_s * len(y_true_exam) + score_i * np.sum(q_image) * weights[-1]) / (len(y_true_exam)+np.sum(q_image)* weights[-1])
    scores["total_weighted_average_bce"] = score_all
    print("{} bce: {:.6f}".format(("total_weighted_average_bce" +" "*50)[:30], score_all))
    return scores

In [10]:
col_preds_b0 = []
for col in col_targets[:-1]:
    col_preds_b0.append("{}_pred_b0".format(col))
col_preds_b2 = []
for col in col_targets[:-1]:
    col_preds_b2.append("{}_pred_b2".format(col))

In [11]:
# calc score of B0
rate = [1, 0]
preds_exam_mean = df_train_exam[col_preds_b0].values * rate[0] + df_train_exam[col_preds_b2].values * rate[1]
preds_image_mean = df_train['{}_qpred_b0'.format(col_targets[-1])].values * rate[0] + df_train['{}_qpred_b2'.format(col_targets[-1])].values * rate[1]
scores = calc_metric(df_train_exam[col_targets[:-1]].values,
                     preds_exam_mean,
                     df_train[col_targets[-1]].values, 
                     preds_image_mean.astype(np.float32),
                     df_train['q_i'].values,
                    )

negative_exam_for_pe           bce: 0.359541, auc: 0.884346
indeterminate                  bce: 0.087319, auc: 0.792349
chronic_pe                     bce: 0.162674, auc: 0.669527
acute_and_chronic_pe           bce: 0.083265, auc: 0.832129
central_pe                     bce: 0.115908, auc: 0.942896
leftsided_pe                   bce: 0.298116, auc: 0.893232
rightsided_pe                  bce: 0.310892, auc: 0.901725
rv_lv_ratio_gte_1              bce: 0.237669, auc: 0.895189
rv_lv_ratio_lt_1               bce: 0.349399, auc: 0.824755
weighted_average_study_bce     bce: 0.203126
q_weighted_average_image_bce   bce: 0.216509, auc: 0.962007
total_weighted_average_bce     bce: 0.209738


In [12]:
# calc score of B0
rate = [0, 1]
preds_exam_mean = df_train_exam[col_preds_b0].values * rate[0] + df_train_exam[col_preds_b2].values * rate[1]
preds_image_mean = df_train['{}_qpred_b0'.format(col_targets[-1])].values * rate[0] + df_train['{}_qpred_b2'.format(col_targets[-1])].values * rate[1]
scores = calc_metric(df_train_exam[col_targets[:-1]].values,
                     preds_exam_mean,
                     df_train[col_targets[-1]].values, 
                     preds_image_mean.astype(np.float32),
                     df_train['q_i'].values,
                    )

negative_exam_for_pe           bce: 0.350100, auc: 0.890913
indeterminate                  bce: 0.086860, auc: 0.793543
chronic_pe                     bce: 0.159901, auc: 0.670996
acute_and_chronic_pe           bce: 0.081502, auc: 0.844675
central_pe                     bce: 0.112595, auc: 0.948231
leftsided_pe                   bce: 0.288365, auc: 0.896944
rightsided_pe                  bce: 0.301904, auc: 0.907232
rv_lv_ratio_gte_1              bce: 0.235148, auc: 0.894903
rv_lv_ratio_lt_1               bce: 0.346014, auc: 0.828440
weighted_average_study_bce     bce: 0.199265
q_weighted_average_image_bce   bce: 0.209543, auc: 0.963150
total_weighted_average_bce     bce: 0.204343


In [13]:
# calc score of mean prediction of B0 and B2
rate = [0.5, 0.5]
preds_exam_mean = df_train_exam[col_preds_b0].values * rate[0] + df_train_exam[col_preds_b2].values * rate[1]
preds_image_mean = df_train['{}_qpred_b0'.format(col_targets[-1])].values * rate[0] + df_train['{}_qpred_b2'.format(col_targets[-1])].values * rate[1]
scores = calc_metric(df_train_exam[col_targets[:-1]].values,
                     preds_exam_mean,
                     df_train[col_targets[-1]].values, 
                     preds_image_mean.astype(np.float32),
                     df_train['q_i'].values,
                    )

negative_exam_for_pe           bce: 0.345601, auc: 0.895471
indeterminate                  bce: 0.084615, auc: 0.812207
chronic_pe                     bce: 0.158846, auc: 0.682470
acute_and_chronic_pe           bce: 0.081380, auc: 0.842780
central_pe                     bce: 0.111196, auc: 0.949153
leftsided_pe                   bce: 0.287404, auc: 0.900646
rightsided_pe                  bce: 0.298846, auc: 0.911071
rv_lv_ratio_gte_1              bce: 0.228831, auc: 0.902324
rv_lv_ratio_lt_1               bce: 0.341484, auc: 0.836057
weighted_average_study_bce     bce: 0.196253
q_weighted_average_image_bce   bce: 0.205954, auc: 0.965569
total_weighted_average_bce     bce: 0.201046


# Solving the conflict of label consistency

In [14]:
# process conflict
def solve_conflict(df_pred_s, df_pred, TH_NEGATIVE=0.5, TH_INDETERMINATE = 0.5, verbose=True):
    index_indeterminate = df_pred_s['indeterminate']>TH_INDETERMINATE
    index_negative = (index_indeterminate==False) & (df_pred_s['negative_exam_for_pe']>TH_NEGATIVE)
    index_positive = (index_indeterminate==False) & (index_negative==False)


    index_negative_and_negative_lte_05 = index_negative & (df_pred_s['negative_exam_for_pe']<=0.5)
    df_pred_s['negative_exam_for_pe'][index_negative_and_negative_lte_05] = 0.5001

    index_indeterminate_and_indeterminate_lte_05 = index_indeterminate & (df_pred_s['indeterminate']<=0.5)
    df_pred_s['indeterminate'][index_indeterminate_and_indeterminate_lte_05] = 0.5001

    index_indeterminate_and_negative_gt_05 = index_indeterminate & (df_pred_s['negative_exam_for_pe']>0.5)
    df_pred_s['negative_exam_for_pe'][index_indeterminate_and_negative_gt_05] = 0.5

    index_negative_and_indeterminate_gt_05 = index_negative & (df_pred_s['indeterminate']>0.5)
    df_pred_s['indeterminate'][index_negative_and_indeterminate_gt_05] = 0.5

    
    index_positive_and_negative_gt_05 = index_positive & (df_pred_s['negative_exam_for_pe']>0.5)
    df_pred_s['negative_exam_for_pe'][index_positive_and_negative_gt_05] = 0.5
    
    index_positive_and_indeterminate_gt_05 = index_positive & (df_pred_s['indeterminate']>0.5)
    df_pred_s['indeterminate'][index_positive_and_indeterminate_gt_05] = 0.5
    
    ################################################
    index_negative_and_rv_lv_ratio_lt_1_gt_05 = (index_positive==False) & (df_pred_s['rv_lv_ratio_lt_1']>0.5)
    df_pred_s['rv_lv_ratio_lt_1'][index_negative_and_rv_lv_ratio_lt_1_gt_05] = 0.5

    index_negative_and_rv_lv_ratio_gte_1_gt_05 = (index_positive==False) & (df_pred_s['rv_lv_ratio_gte_1']>0.5)
    df_pred_s['rv_lv_ratio_gte_1'][index_negative_and_rv_lv_ratio_gte_1_gt_05] = 0.5

    index_negative_and_central_pe_gt_05 = (index_positive==False) & (df_pred_s['central_pe']>0.5)

    index_negative_and_rightsided_pe_gt_05 = (index_positive==False) & (df_pred_s['rightsided_pe']>0.5)
    df_pred_s['rightsided_pe'][index_negative_and_rightsided_pe_gt_05] = 0.5

    index_negative_and_leftsided_pe_gt_05 = (index_positive==False) & (df_pred_s['leftsided_pe']>0.5)
    df_pred_s['leftsided_pe'][index_negative_and_leftsided_pe_gt_05] = 0.5

    index_negative_and_chronic_pe_gt_05 = (index_positive==False) & (df_pred_s['chronic_pe']>0.5)
    df_pred_s['chronic_pe'][index_negative_and_chronic_pe_gt_05] = 0.5

    index_negative_and_acute_and_chronic_pe_gt_05 = (index_positive==False) & (df_pred_s['acute_and_chronic_pe']>0.5)
    df_pred_s['acute_and_chronic_pe'][index_negative_and_acute_and_chronic_pe_gt_05] = 0.5

    ################################################
    index_positive_and_rv_gte_lv = index_positive & (df_pred_s['rv_lv_ratio_lt_1']<=df_pred_s['rv_lv_ratio_gte_1'])
    index_positive_and_rv_lt_lv = index_positive & (df_pred_s['rv_lv_ratio_lt_1']>df_pred_s['rv_lv_ratio_gte_1'])

    index_positive_and_rv_gte_lv_and_rv_lv_ratio_gte_1_lte_05 =\
        (index_positive_and_rv_gte_lv) & (df_pred_s['rv_lv_ratio_gte_1']<=0.5)
    df_pred_s['rv_lv_ratio_gte_1'][index_positive_and_rv_gte_lv_and_rv_lv_ratio_gte_1_lte_05] = 0.5001

    index_positive_and_rv_gte_lv_and_rv_lv_ratio_lt_1_gt_05 =\
        (index_positive_and_rv_gte_lv) & (df_pred_s['rv_lv_ratio_lt_1']>0.5)
    df_pred_s['rv_lv_ratio_lt_1'][index_positive_and_rv_gte_lv_and_rv_lv_ratio_lt_1_gt_05] = 0.5

    index_positive_and_rv_lt_lv_and_rv_lv_ratio_lt_1_lte_05 =\
        (index_positive_and_rv_lt_lv) & (df_pred_s['rv_lv_ratio_lt_1']<=0.5)
    df_pred_s['rv_lv_ratio_lt_1'][index_positive_and_rv_lt_lv_and_rv_lv_ratio_lt_1_lte_05] = 0.5001

    index_positive_and_rv_lt_lv_and_rv_lv_ratio_gte_1_gt_05 =\
        (index_positive_and_rv_lt_lv) & (df_pred_s['rv_lv_ratio_gte_1']>0.5)
    df_pred_s['rv_lv_ratio_gte_1'][index_positive_and_rv_lt_lv_and_rv_lv_ratio_gte_1_gt_05] = 0.5

    index_positive_and_central_is_greatest = index_positive & (df_pred_s['central_pe']>=df_pred_s['rightsided_pe']) & (df_pred_s['central_pe']>=df_pred_s['leftsided_pe'])
    index_positive_and_right_is_greatest = index_positive & (index_positive_and_central_is_greatest==False) & (df_pred_s['rightsided_pe']>=df_pred_s['leftsided_pe'])
    index_positive_and_left_is_greatest = index_positive & (index_positive_and_central_is_greatest==False) & (index_positive_and_right_is_greatest==False) 


    index_positive_and_central_is_greatest_and_central_pe_lte_05 = (index_positive_and_central_is_greatest) & (df_pred_s['central_pe']<=0.5)
    df_pred_s['central_pe'][index_positive_and_central_is_greatest_and_central_pe_lte_05] = 0.5001

    index_positive_and_right_is_greatest_and_rightsided_pe_lte_05 = (index_positive_and_right_is_greatest) & (df_pred_s['rightsided_pe']<=0.5)
    df_pred_s['rightsided_pe'][index_positive_and_right_is_greatest_and_rightsided_pe_lte_05] = 0.5001

    index_positive_and_left_is_greatest_and_leftsided_pe_lte_05 = (index_positive_and_left_is_greatest) & (df_pred_s['leftsided_pe']<=0.5)
    df_pred_s['leftsided_pe'][index_positive_and_left_is_greatest_and_leftsided_pe_lte_05] = 0.5001

     # acute_and_chronic_pe and chronic_pe: only one of them can have p > 0.5; neither having p > 0.5 is allowed.
    index_double_positive = index_positive & (df_pred_s['chronic_pe']>0.5) & (df_pred_s['acute_and_chronic_pe']>0.5)

    index_double_positive_and_chronic_lte_acute_and_chronic = index_double_positive & (df_pred_s['chronic_pe']<=df_pred_s['acute_and_chronic_pe'])
    df_pred_s['chronic_pe'][index_double_positive_and_chronic_lte_acute_and_chronic] = 0.5

    index_double_positive_and_chronic_gt_acute_and_chronic = index_double_positive & (df_pred_s['chronic_pe']>df_pred_s['acute_and_chronic_pe'])
    df_pred_s['acute_and_chronic_pe'][index_double_positive_and_chronic_gt_acute_and_chronic] = 0.5

    ################################################
    df_pred_s['positive'] = 0
    df_pred_s['positive'][index_positive] = 1
    df_pred2 = pd.merge(df_pred, df_pred_s[[col_groupby, 'positive']], on=col_groupby, how='left')

    df_agg = df_pred.groupby(col_groupby)['pe_present_on_image'].agg('max').reset_index()
    df_agg.columns = [col_groupby, 'pe_present_on_image_pred_max']
    df_pred2 = pd.merge(df_pred2, df_agg, on=col_groupby, how='left')
    df_pred2['peak'] = df_pred2['pe_present_on_image']==df_pred2['pe_present_on_image_pred_max']
    # df_tmp = df_s_p[[col_groupby]]
    # df_tmp['positive'] = True

    index_positive_i = df_pred2['positive']==1

    index_negative_and_pe_present_on_image_gt_05_i = (index_positive_i==False) & (df_pred2['pe_present_on_image']>0.5)
    df_pred['pe_present_on_image'][index_negative_and_pe_present_on_image_gt_05_i] = 0.5

    index_positive_and_peak_and_pe_present_on_image_lte_05_i = index_positive_i & (df_pred2['peak']) & (df_pred2['pe_present_on_image']<=0.5)
    df_pred['pe_present_on_image'][index_positive_and_peak_and_pe_present_on_image_lte_05_i] = 0.5001
   
    if verbose:
        print("num study", len(df_pred_s))
        print("num image", len(df_pred))
        print("split to 3 classes")
        print(" num predicted_as_negative:", index_negative.sum())
        print(" num predicted_as_indeterminate:", index_indeterminate.sum())
        print(" num predicted_as_positive:", index_positive.sum())
        print("process 3 class conflict")
        print(" num predicted_as_negative and negative<=0.5:", index_negative_and_negative_lte_05.sum())
        print(" num predicted_as_indeterminate and indeterminate<=0.5:", index_indeterminate_and_indeterminate_lte_05.sum())

        print(" num predicted_as_indeterminate and negative_exam_for_pe>0.5:", index_indeterminate_and_negative_gt_05.sum())
        print(" num predicted_as_negative and indeterminate>0.5:", index_negative_and_indeterminate_gt_05.sum())
        print(" num predicted_as_positive and negative_exam_for_pe>0.5:", index_positive_and_negative_gt_05.sum())
        print(" num predicted_as_positive and indeterminate>0.5:", index_positive_and_indeterminate_gt_05.sum())
      
        print("process negative case")
        print(" num predicted_as_not_positive and rv_lv_ratio_lt_1>0.5:", index_negative_and_rv_lv_ratio_lt_1_gt_05.sum())
        print(" num predicted_as_not_positive and rv_lv_ratio_gte_1>0.5:", index_negative_and_rv_lv_ratio_gte_1_gt_05.sum())
        print(" num predicted_as_not_positive and central_pe>0.5:", index_negative_and_central_pe_gt_05.sum())
        print(" num predicted_as_not_positive and central_pe>0.5:", index_negative_and_rightsided_pe_gt_05.sum())
        print(" num predicted_as_not_positive and leftsided_pe>0.5:", index_negative_and_leftsided_pe_gt_05.sum())
        print(" num predicted_as_not_positive and chronic_pe>0.5:", index_negative_and_chronic_pe_gt_05.sum())
        print(" num predicted_as_not_positive and acute_and_chronic_pe>0.5:", index_negative_and_acute_and_chronic_pe_gt_05.sum())

        print("process positive case")
        print(" num predicted_as_positive and rv_lv_ratio_lt_1<=rv_lv_ratio_gte_1:", index_positive_and_rv_gte_lv.sum())
        print(" num predicted_as_positive and rv_lv_ratio_lt_1>rv_lv_ratio_gte_1:", index_positive_and_rv_lt_lv.sum())
        print(" num predicted_as_positive and (rv_lv_ratio_lt_1<=rv_lv_ratio_gte_1) and (rv_lv_ratio_gte_1<=0.5): ",
               index_positive_and_rv_gte_lv_and_rv_lv_ratio_gte_1_lte_05.sum())
        print(" num predicted_as_positive and (rv_lv_ratio_lt_1<=rv_lv_ratio_gte_1) and rv_lv_ratio_lt_1>0.5: ",
               index_positive_and_rv_gte_lv_and_rv_lv_ratio_lt_1_gt_05.sum())
        print(" num predicted_as_positive and (rv_lv_ratio_lt_1>rv_lv_ratio_gte_1) and rv_lv_ratio_lt_1<=0.5: ",
               index_positive_and_rv_lt_lv_and_rv_lv_ratio_lt_1_lte_05.sum())
        print(" num predicted_as_positive and (rv_lv_ratio_lt_1>rv_lv_ratio_gte_1) and rv_lv_ratio_gte_1>0.5: ",
               index_positive_and_rv_lt_lv_and_rv_lv_ratio_gte_1_gt_05.sum())
        print(" num predicted_as_positive and central is greatest:", index_positive_and_central_is_greatest.sum())
        print(" num predicted_as_positive and right is greatest:", index_positive_and_right_is_greatest.sum())
        print(" num predicted_as_positive and left is greatest:", index_positive_and_left_is_greatest.sum())
        print(" num predicted_as_positive and central is greatest and central_pe<=0.5:", index_positive_and_central_is_greatest_and_central_pe_lte_05.sum())
        print(" num predicted_as_positive and right is greatest and rightsided_pe<=0.5:", index_positive_and_right_is_greatest_and_rightsided_pe_lte_05.sum())
        print(" num predicted_as_positive and left is greatest and leftsided_pe<=0.5:", index_positive_and_left_is_greatest_and_leftsided_pe_lte_05.sum())
        print(" num both chronic_pe and acute_and_chronic_pe is positive:", index_double_positive.sum())
        print(" num both chronic_pe and acute_and_chronic_pe is positive and chronic<=acute_and_chronic:", index_double_positive_and_chronic_lte_acute_and_chronic.sum())
        print(" num both chronic_pe and acute_and_chronic_pe is positive and chronic>acute_and_chronic:", index_double_positive_and_chronic_gt_acute_and_chronic.sum())

        print("process image level")
        print(" num img of predicted_as_positive:", index_positive_i.sum())
        print(" num img of predicted_as_negative:", (index_positive_i==0).sum())
        print(" num img of peak:", df_pred2['peak'].sum())
        print(" num img of predicted_as_negative and pe_present_on_image>0.5:", index_negative_and_pe_present_on_image_gt_05_i.sum())
        print(" num img of predicted_as_positive and peak and pe_present_on_image<=0.5:", index_positive_and_peak_and_pe_present_on_image_lte_05_i.sum())

    return df_pred_s, df_pred


In [15]:
# get the label-consistancy-solved prediction
rate = [0.5, 0.5]
preds_exam_mean = df_train_exam[col_preds_b0].values * rate[0] + df_train_exam[col_preds_b2].values * rate[1]
preds_image_mean = df_train['{}_qpred_b0'.format(col_targets[-1])].values * rate[0] + df_train['{}_qpred_b2'.format(col_targets[-1])].values * rate[1]

df_tmp_exam = df_train_exam[[col_groupby]]
for i, col in enumerate(col_targets[:-1]):
    df_tmp_exam[col] = preds_exam_mean[:,i]
df_tmp = df_train[[col_groupby]]
df_tmp[col_targets[-1]] = preds_image_mean
df_pred_exam_const, df_pred_const = solve_conflict(
    df_tmp_exam,
    df_tmp, 
    TH_NEGATIVE=0.5, TH_INDETERMINATE = 0.5, verbose=True)

num study 7279
num image 1790594
split to 3 classes
 num predicted_as_negative: 5607
 num predicted_as_indeterminate: 4
 num predicted_as_positive: 1668
process 3 class conflict
 num predicted_as_negative and negative<=0.5: 0
 num predicted_as_indeterminate and indeterminate<=0.5: 0
 num predicted_as_indeterminate and negative_exam_for_pe>0.5: 0
 num predicted_as_negative and indeterminate>0.5: 0
 num predicted_as_positive and negative_exam_for_pe>0.5: 0
 num predicted_as_positive and indeterminate>0.5: 0
process negative case
 num predicted_as_not_positive and rv_lv_ratio_lt_1>0.5: 0
 num predicted_as_not_positive and rv_lv_ratio_gte_1>0.5: 0
 num predicted_as_not_positive and central_pe>0.5: 0
 num predicted_as_not_positive and central_pe>0.5: 0
 num predicted_as_not_positive and leftsided_pe>0.5: 0
 num predicted_as_not_positive and chronic_pe>0.5: 0
 num predicted_as_not_positive and acute_and_chronic_pe>0.5: 0
process positive case
 num predicted_as_positive and rv_lv_ratio_lt_1<=

In [16]:
# calc the CV score of the label-consistancy-solved prediction
scores = calc_metric(df_train_exam[col_targets[:-1]].values,
                     df_pred_exam_const[col_targets[:-1]].values,
                     df_train[col_targets[-1]].values, 
                     df_pred_const[col_targets[-1]].values,
                     df_train['q_i'].values,
                    )

negative_exam_for_pe           bce: 0.345601, auc: 0.895471
indeterminate                  bce: 0.084615, auc: 0.812207
chronic_pe                     bce: 0.158846, auc: 0.682470
acute_and_chronic_pe           bce: 0.081380, auc: 0.842780
central_pe                     bce: 0.111196, auc: 0.949153
leftsided_pe                   bce: 0.287363, auc: 0.900652
rightsided_pe                  bce: 0.298690, auc: 0.911001
rv_lv_ratio_gte_1              bce: 0.228942, auc: 0.902246
rv_lv_ratio_lt_1               bce: 0.342175, auc: 0.835369
weighted_average_study_bce     bce: 0.196321
q_weighted_average_image_bce   bce: 0.206749, auc: 0.965775
total_weighted_average_bce     bce: 0.201473
